In [ ]:
!pip install gradio

from tqdm import tqdm
from google.colab import files
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import gradio as gr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 10.6 MB/s 

In [ ]:
# VGG
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        self.chosen_features = ["0", "5", "10", "19", "28"]
        self.model = models.vgg19(pretrained=True).features[:29]

    def forward(self, x):
        features = []
        for layer_num, layer in enumerate(self.model):
            x = layer(x)
            if str(layer_num) in self.chosen_features:
                features.append(x)
        return features

In [ ]:
def style_transfer(original_image, style_image, total_steps, alpha, beta, learning_rate, imsize, optimizer_name):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    loader = transforms.Compose([
        transforms.Resize((imsize, imsize)),
        transforms.ToTensor(),
    ])
    model = VGG().to(device).eval()

    def load_image_gradio(image):
        original_image = Image.open(image)
        original_height, original_width = original_image.size
        image = loader(original_image).unsqueeze(0)
        return image.to(device), (original_width, original_height)

    original_img, original_size = load_image_gradio(original_image)
    style_img, _ = load_image_gradio(style_image)
    generated = original_img.clone().requires_grad_(True)

    #generated = torch.randn(original_img.shape, device=device, requires_grad=True)


    # Optimizer seçimi
    if optimizer_name == 'RMSprop':
        optimizer = optim.RMSprop([generated], lr=learning_rate)
    elif optimizer_name == 'Adam':
        optimizer = optim.Adam([generated], lr=learning_rate)
    elif optimizer_name == 'Adagrad':
        optimizer = optim.Adagrad([generated], lr=learning_rate)
    elif optimizer_name == 'Adadelta':
        optimizer = optim.Adadelta([generated], lr=learning_rate)
    elif optimizer_name == 'Adamax':
        optimizer = optim.Adamax([generated], lr=learning_rate)
    elif optimizer_name == 'NAdam':
        optimizer = optim.NAdam([generated], lr=learning_rate)
    else:
        raise ValueError(f"Unsupported optimizer: {optimizer_name}")

    total_loss_list = []
    style_loss_list = []
    content_loss_list = []

    for step in tqdm(range(total_steps)):
        generated_features = model(generated)
        original_img_features = model(original_img)
        style_features = model(style_img)

        style_loss = original_loss = 0

        for gen_feature, orig_feature, style_feature in zip(
            generated_features, original_img_features, style_features
        ):
            batch_size, channel, height, width = gen_feature.shape
            original_loss += torch.mean((gen_feature - orig_feature) ** 2)

            G = gen_feature.view(channel, height * width).mm(
                gen_feature.view(channel, height * width).t()
            )
            A = style_feature.view(channel, height * width).mm(
                style_feature.view(channel, height * width).t()
            )
            style_loss += torch.mean((G - A) ** 2)

        total_loss = alpha * original_loss + beta * style_loss
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        total_loss_list.append(total_loss.item())
        style_loss_list.append(style_loss.item())
        content_loss_list.append(original_loss.item())

        if step % 500 == 0:
            tqdm.write(f"Step {step}, Total Loss: {total_loss.item()}")

    def format_ticks(value, pos):
        if value >= 1_000_000:
            return f'{value / 1_000_000:.1f}M'
        elif value >= 1_000:
            return f'{value / 1_000:.1f}k'
        else:
            return f'{value:g}'


    shifted_steps = range(100, total_steps)

    plt.figure(figsize=(24, 8))
    plt.plot(shifted_steps, total_loss_list[100:], label='Total Loss')
    plt.xlabel('Step')
    plt.ylabel('Total Loss')
    plt.title('Total Loss Değişimi')
    plt.legend()
    plt.gca().yaxis.set_major_formatter(FuncFormatter(format_ticks))
    plt.xticks(range(0, total_steps + 1, max(1, total_steps // 20)))
    plt.savefig('total_loss_plot.png')
    plt.close()

    plt.figure(figsize=(24, 8))
    plt.plot(shifted_steps, style_loss_list[100:], label='Style Loss', color='orange')
    plt.xlabel('Step')
    plt.ylabel('Style Loss')
    plt.title('Style Loss Değişimi')
    plt.legend()
    plt.gca().yaxis.set_major_formatter(FuncFormatter(format_ticks))
    plt.xticks(range(0, total_steps + 1, max(1, total_steps // 20)))
    plt.savefig('style_loss_plot.png')
    plt.close()

    plt.figure(figsize=(24, 8))
    plt.plot(content_loss_list[100:], label='Content Loss', color='green')
    plt.xlabel('Step')
    plt.ylabel('Content Loss')
    plt.title('Content Loss Değişimi')
    plt.legend()
    plt.gca().yaxis.set_major_formatter(FuncFormatter(format_ticks))
    plt.xticks(range(0, total_steps + 1, max(1, total_steps // 20)))
    plt.savefig('content_loss_plot.png')
    plt.close()

    generated = transforms.Resize(original_size)(generated.squeeze(0).cpu()).unsqueeze(0)
    save_image(generated, "generated.png")
    return "generated.png", 'total_loss_plot.png', 'style_loss_plot.png', 'content_loss_plot.png'

interface = gr.Interface(
    fn=style_transfer,
    inputs=[
        gr.components.Image(type="filepath", label="Orijinal Resim"),
        gr.components.Image(type="filepath", label="Stil Resmi"),
        gr.components.Slider(minimum=100, maximum=10000, step=100, label="Toplam Adım Sayısı", value=3000),
        gr.components.Number(minimum=0, maximum=1, step=0.01, label="Alpha Katsayısı", value=0.01),
        gr.components.Number(minimum=0, maximum=10, step=0.1, label="Beta Katsayısı", value=1),
        gr.components.Number(minimum=0.0001, maximum=0.99999, step=0.0001, label="Öğrenme Oranı", value=0.001),
        gr.components.Number(minimum=128, maximum=4096, step=1, label="Image Size", value=512),
        gr.components.Dropdown(choices=['RMSprop' ,'Adam' ,'Adagrad', 'Adadelta', 'Adamax', 'NAdam'], value='RMSprop')
        ],
    outputs=[
        gr.components.Image(type="filepath", label="Oluşturulan Resim"),
        gr.components.Image(type="filepath", label="Total Loss Grafiği"),
        gr.components.Image(type="filepath", label="Style Loss Grafiği"),
        gr.components.Image(type="filepath", label="Content Loss Grafiği")
    ],
    title="Neural Style Transfer",
    description="Yüklediğiniz iki resmi kullanarak bir neural style transfer gerçekleştirin."
)

interface.launch(debug=True, inline=False)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://d8534ebf32c16dfd60.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:02<00:00, 226MB/s]
  0%|          | 2/3000 [00:02<50:52,  1.02s/it]  

Step 0, Total Loss: 269837760.0


 17%|█▋        | 502/3000 [01:10<05:46,  7.20it/s]

Step 500, Total Loss: 1443364.25


...

Step 2000, Total Loss: 559341.875


 83%|████████▎ | 2502/3000 [05:54<01:11,  7.00it/s]

Step 2500, Total Loss: 486081.78125


100%|██████████| 3000/3000 [07:05<00:00,  7.05it/s]


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d8534ebf32c16dfd60.gradio.live
